# Using pre-trained word embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/05<br>
**Last modified:** 2020/05/05<br>
**Description:** Text classification on the Newsgroup20 dataset using pre-trained GloVe word embeddings.

##설정

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 소개

이 예에서는 사전 훈련된 단어 임베딩을 사용하는 텍스트 분류 모델을 훈련하는 방법을 보여줍니다.

20개의 다른 주제 범주에 속하는 20,000개의 게시판 메시지 세트인 Newsgroup20 데이터 세트로 작업하겠습니다.

사전 훈련된 단어 임베딩의 경우 GloVe 임베딩을 사용 합니다.

[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

GloVe(Global Word Vectors) : 스탠포드대학에서 개발한 단어 임베딩 기법, 
1) 목표 : 임베딩된 단어 벡터간 유사도 측정을 수월하게 하면서 말뭉치 전체의 통계정보를 좀 더 잘 반영해 보자

2) 특징 :처음 코퍼스를 통해 단어별 동시 출현 빈도를 조사해 출현 빈도 행렬을 만들고, 이후에는 해당 행렬을 통해 동시 출현 빈도를 근사하기 때문에 학습 속도가빠르다.

3) 방법 :
-  우선 학습 말뭉치 대상으로 단어-문맥 행렬A를 만드는것에서부터 학습을 시작한다. 
- 이후 목적함수를 최소화 하는 임베딩 벡터를 찾기 위해 행렬 분해를 수행해야 한다. 처음에는 행렬, U, V 랜덤으로 초기화 한후 수식을 최소화 하는 방향으로 U, V를 조금씩 업데이트 해나간다. 
- 학습손실이 더 줄지 않거나 정해진 스텝수만큼 학습했을 경우 학습을 종료한다. 학습이 끝나면 U를 단어 임베딩으로 쓸수 있다. 
- 전역 단어-단어 동시 발생 행렬의 0이 아닌 항목에 대해 학습되며, 이 행렬은 주어진 말뭉치에서 단어가 서로 얼마나 자주 동시 발생하는지 표를 작성


- 동시등장 행렬 :학습 말뭉치에서 동시에 등장한 단어의 빈도를 세어서 전체 말뭉치의 단어 개수로 나눠준 행렬

![동시발생 행렬](https://user-images.githubusercontent.com/70866993/130650771-704a37d4-ecbd-4927-9cc2-02aa7f488719.PNG)


* 추론 기반 기법과 통계 기반 기법을 융합한 GloVe

추론 기반 기법 : 말뭉치 일부를 여러 번 보며 학습(미니배치 학습). 통계 기반 기법이 주로 단어의 유사성을 인코딩하는 것에서 나아가 한층 복잡한 단어 사이의 패턴까지도 인코딩 할 수 있음(king-man+woman=queen 가능)

![추론기반과 통계기반 융합](https://user-images.githubusercontent.com/70866993/130650810-ab1c5761-2fcb-4392-921e-9a9e6da5b47b.PNG)


## Newsgroup20 데이터 다운로드

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

## 데이터 살펴보기

In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir) # 입력 경로 내의 모든 파일과 폴더명 리스트 반환.
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")  #숨김파일 .keras/datasets/20_newgroup/comp.graphics 세로 오름차순정렬된 filenames 확인됨
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])  # 랜덤 값인것처럼 배치되어 있음

Number of directories: 20
Directory names: ['talk.religion.misc', 'talk.politics.mideast', 'sci.crypt', 'misc.forsale', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware', 'sci.space', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'rec.sport.hockey', 'talk.politics.misc', 'talk.politics.guns', 'rec.sport.baseball', 'alt.atheism', 'rec.autos', 'comp.graphics', 'sci.med', 'comp.windows.x', 'rec.motorcycles', 'soc.religion.christian']
Number of files in comp.graphics: 1000
Some example filenames: ['38596', '38770', '38333', '38432', '38744']


다음은 한 파일에 포함된 내용의 예입니다.

In [4]:
print(open(data_dir / "comp.graphics" / "38987").read()) 

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



보시다시피 명시적으로(첫 번째 줄은 말 그대로 범주 이름임) 또는 암시적으로(예: 파일을 통해) 파일의 범주를 누출하는 헤더 줄이 있습니다 Organization. 헤더를 제거합시다.
- Organization: Rensselaer Polytechnic Institute, Troy, NY. ->조직: 렌셀러 폴리테크닉 인스티튜트, 트로이, 뉴욕.

In [5]:
# 샘플 목록에 for 문을 활용하여 라벨, 클래스명, 클래스 인덱스 형식으로 정리, 클래스명, 샘플수 확인
samples = []
labels = []
class_names = []
class_index = 0

for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname) # dirname 추가
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

실제로 예상되는 파일 수가 없는 카테고리가 하나 있지만 적은 수라 균형 잡힌 분류 문제로서 지장이 없다.

## 데이터를 섞고 훈련 및 검증 세트로 분할

In [6]:
# Shuffle the data

seed = 1337  # seed 어떤 값을 넣던지 별 차이 없이 numpy로 하여금 서로 다른 유사난수 생성
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# 훈련 및 검증 분할 추출
validation_split = 0.2  #유효성 검사를 위해 데이터의 20%를 사용
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples] 
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

<b>np.random.RandomState()Permalink</b>
- 특정 seed를 가지는 np.random.RandomState()를 만들어주고, 여기서부터 이 object에 접근하여 난수를 생성

<b>np.random.random()</b>
- numpy에 존재하는 random generator에 직접 접근하여, 난수를 생성

## 어휘 인덱싱 만들기

<b>TextVectorization</b>
- 데이터 세트에서 찾은 어휘를 인덱싱 하는 데 사용, 추후 동일 레이어 인스턴트사용 샘플 벡터화
- 텍스트 벡터화 : 자연어 처리에서 기계가 문자를 이해할수 있도록 수치화 해주는 과정

텍스트 벡터화중 원-핫인코딩
![텍스트벡터화](https://user-images.githubusercontent.com/70866993/130627843-f36086d1-29db-4e50-afb0-be9d7de638b1.PNG)


In [7]:
from tensorflow.keras.layers import TextVectorization

# 레이어의 상위 2만단어 고려, 시퀀스를 자르거나 채워서 실제로 200개 토큰 길이 맞추기
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)

# 대용량의 데이터에서 배치 단위로 데이터를 가져오기 위한 데이터셋 제너레이터를 생성하는 코드
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)

vectorizer.adapt(text_ds)

이를 통해 사용된 계산 어휘를 검색 할 수 있습니다.  `vectorizer.get_vocabulary()`. 상위 5개 단어를 인쇄 해보기

In [8]:
# 상위 5개 어휘 검색

vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

테스트 문장을 벡터화합시다

In [9]:
# 테스트 문장 벡터화하기

output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3315, 1823,   15,    2, 6018])

In [10]:
# 단어를 인덱스에 매핑

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [11]:
# 테스트 문장에 대해 위와 동일한 인코딩 확인  ([9]번화 동일한 결과를 얻었다.)

test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3315, 1823, 15, 2, 6018]

## 사전 훈련된 단어 임베딩 로드

- 사전 훈련된 GloVe 임베딩(822M zip 파일)을 다운로드 하기

- 다음 명령을 실행 

```
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
```

아카이브에는 50차원, 100차원, 200차원, 300차원 등 다양한 크기의 텍스트로 인코딩된 벡터가 포함되어 있다.
우리는 100D를 사용할 것입니다.

NumPy 벡터 표현에 단어(문자열)를 매핑하는 딕셔너리를 만들어 보겠습니다.

In [12]:
#  단어(문자열)를 매핑하는 딕셔너리 만들기

path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "aiffel/nlp/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1) #단어,계수(변수에 입정하게 곱해진 상수)= 라인.분할(최대분할=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("%s 워드 벡터를 찾았습니다.." % len(embeddings_index))

400000 워드 벡터를 찾았습니다..


In [13]:
# Keras에서 사용할 수 있는 해당 임베딩 매트릭스를 준비

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# 임베딩 메트릭스 준비 
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word) #단어(key) 해당되는 임베딩 벡터의 값을  embedding_vector 저장
    if embedding_vector is not None:
        # 임베딩 인덱스에서 찾을 수 없는 단어는 모두 0입니다.
        # 여기에는 "padding" 및 "OOV"(=모르는단어)에 대한 표현이 포함됩니다.
        embedding_matrix[i] = embedding_vector  # embedding_vector의 값을 `i`의 단어 벡터와 맵핑되는 vectorizer` 어휘 인덱스 행에 삽입
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses)) 

print('변환된 {} 단어가 있습니다.'.format(int(hits)))  
print('실패한 {} 단어가 있습니다.'.format(int(misses)))  

Converted 17998 words (2002 misses)
변환된 17998 단어가 있습니다.
실패한 2002 단어가 있습니다.


- 인덱스 0은 패딩용, 인덱스 1은 "어휘 외" 토큰용으로 사용됨

In [14]:
# 사전 훈련된 단어 임베딩 행렬을 '임베딩' 층 만들기
# 임베딩을 고정된 상태로 유지하기 위해 `trainable=False`를 설정

from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

- 사전 훈련된 단어 임베딩을 사용하지 않고 Embedding계층을 처음부터 초기화하고
  훈련 중에 가중치를 학습함으로써 얼마나 잘 수행했는지 테스트할 수도 있습니다 . 
  Embedding레이어를 다음 으로 교체하기만 하면 됩니다 .

<mark>Skip-Gram with Negative Sampling, SGNS</mark>
- SGNS는 다음과 같이 중심 단어와 주변 단어가 모두 입력이 되고, 이 두 단어가 실제로 윈도우 크기 내에 존재하는 이웃 관계인지 그 확률을 예측

1) 기존 데이터셋 
![1](https://user-images.githubusercontent.com/70866993/130656478-a08f4316-0915-40ca-adb5-0d106cc5fda8.PNG)

2) 이웃 관계인 경우 레이블을 1(중심단어), 아닌 경우 레이블 0(주변단어)으로 변경
![2](https://user-images.githubusercontent.com/70866993/130656480-a04525e6-70ae-418f-813b-2da1ae24e303.PNG)

3) 중심단어와 주변 단어의 내적값을 이 모델의 예측값으로 하고, 레이블과의 오차로부터 역전파하여 중심 단어와 주변 단어의 임베딩 벡터값 업데이트 
![3](https://user-images.githubusercontent.com/70866993/130656500-eb249330-ee06-44f9-a326-809176ef0d49.PNG)



## 모델 구축

전역 최대 풀링과 끝에 분류기가 있는 간단한 1D convnet.

In [15]:
# 분류 문제 해결을 위한 1D convnet 훈련

from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x) #입력과 출력을 각각 연결해주는 가중치 포함,은닉층활성화함수
x = layers.Dropout(0.5)(x) #랜덤으로 0으로 초기화해서 오버피팅 방지
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048 

## 모델구축

먼저 문자열 목록 데이터를 정수 인덱스의 NumPy 배열로 변환합니다.

In [16]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [17]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 9s 64ms/step - loss: 2.7164 - acc: 0.1260 - val_loss: 2.1146 - val_acc: 0.3006
Epoch 2/20
125/125 [==============================] - 8s 62ms/step - loss: 1.9717 - acc: 0.3220 - val_loss: 1.5746 - val_acc: 0.4676
Epoch 3/20
125/125 [==============================] - 8s 62ms/step - loss: 1.5502 - acc: 0.4711 - val_loss: 1.3385 - val_acc: 0.5369
Epoch 4/20
125/125 [==============================] - 8s 62ms/step - loss: 1.2982 - acc: 0.5528 - val_loss: 1.1856 - val_acc: 0.5994
Epoch 5/20
125/125 [==============================] - 8s 63ms/step - loss: 1.1293 - acc: 0.6096 - val_loss: 1.0611 - val_acc: 0.6369
Epoch 6/20
125/125 [==============================] - 8s 63ms/step - loss: 1.0015 - acc: 0.6566 - val_loss: 1.0484 - val_acc: 0.6507
Epoch 7/20
125/125 [==============================] - 8s 63ms/step - loss: 0.8847 - acc: 0.6914 - val_loss: 1.0803 - val_acc: 0.6519
Epoch 8/20
125/125 [==============================] - 8s 63ms/step - 

- softmax 분류를 수행하기 때문에 다중 분류 손실 함수인 sparse_categorical_crossentropy 사용
- 이 함수는 integer type 클래스 -> one-hot encoding하지 않고 정수 형태로 label(y)을 넣어줍니다.
- 이 함수는 한 샘플에 여러 클래스가 있거나 label이 soft 확률일 경우 사용
-  optimizer의 rmsprop : 이전 누적치, 현재 누적치를 감마(0과1사이)1/2로 잡으면 딱 평균화(중간)되어 AdaGrad보다 속도가 빠름

## 종단 간 모델 내보내기

임의의 문자열을 입력으로 받는 인덱스 시퀀스 model 객체를 내보낼수 있습니다.
모델을 훨씬더 가용성 있게 만들것 입니다. 

In [18]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

'comp.graphics'

## 참고자료

1. 추론 기반 기법과 신경망 https://settlelib.tistory.com/48
2. 동시등장확률 자세히 보기 : https://wikidocs.net/22885
3. GloVe Model : https://insighting.tistory.com/10
4. Negative Sampling : https://wikidocs.net/69141